In [5]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tahtan
# Import Sionna
import sys
sys.path.append('../')
import sionna

# try:
#     import sionna
# except ImportError as e:
#     # Install Sionna if package is not already installed
#     import os
#     os.system("pip install sionna")
#     import sionna

import tensorflow as tf
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible results

# Load the required Sionna components
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver, CarrierConfig, PUSCHDMRSConfig,\
                        TBConfig, PUSCHPilotPattern, TBEncoder, PUSCHPrecoder, LayerMapper, LayerDemapper, check_pusch_configs,\
                        TBDecoder, PUSCHLSChannelEstimator
from sionna.nr.utils import generate_prng_seq
from sionna.channel import AWGN, RayleighBlockFading, OFDMChannel, TimeChannel, time_lag_discrete_time_channel
from sionna.channel.utils import * 
from sionna.channel.tr38901 import Antenna, AntennaArray, UMi, UMa, RMa, TDL, CDL
from sionna.channel import gen_single_sector_topology as gen_topology
from sionna.utils import compute_ber, ebnodb2no, sim_ber, array_to_hash, create_timestamped_folders, b2b, f2f, BinarySource
from sionna.ofdm import KBestDetector, LinearDetector, MaximumLikelihoodDetector,\
        LSChannelEstimator, LMMSEEqualizer, RemoveNulledSubcarriers, ResourceGridDemapper,\
        ResourceGrid, ResourceGridMapper, OFDMModulator
from sionna.mimo import StreamManagement
from sionna.mapping import Mapper, Demapper


In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
# from bs4 import BeautifulSoup
import pickle
from collections import namedtuple
import json
from tqdm.notebook import tqdm
import itertools
import io

## A Hello World Example

Let us start with a simple "Hello, World!" example in which we will simulate PUSCH transmissions from a single transmitter to a single receiver over an AWGN channel.

Hopefully you have enjoyed this tutorial on Sionna's 5G NR PUSCH module!

Please have a look at the [API documentation](https://nvlabs.github.io/sionna/api/sionna.html) of the various components or the other available [tutorials](https://nvlabs.github.io/sionna/tutorials.html) to learn more.

In [7]:
# slots = [None]*20
# for i in [4,5,14,15]:
#     slots[i] = df[(df['nSlot'] == i) & (df['Esno_db'] == 6.)].iloc[0]

In [8]:
class MyPUSCHConfig(PUSCHConfig):
    def __init__(self):
        super().__init__(
            carrier_config=CarrierConfig(
                n_cell_id=0,
                cyclic_prefix="normal",
                subcarrier_spacing=30,
                n_size_grid=273,
                n_start_grid=0,
                slot_number=4,
                frame_number=0
            ),
            pusch_dmrs_config=PUSCHDMRSConfig(
                config_type=1,
                length=1,
                additional_position=1,
                dmrs_port_set=[0],
                n_id=0,
                n_scid=0,
                num_cdm_groups_without_data=2,
                type_a_position=2
            ),
            tb_config=TBConfig(
                channel_type='PUSCH',
                n_id=0,
                mcs_table=1,
                mcs_index=3
            ),
            mapping_type='A',
            n_size_bwp=4,
            n_start_bwp=0,
            num_layers=1,
            num_antenna_ports=1,
            precoding='non-codebook',
            tpmi=0,
            transform_precoding=False,
            n_rnti=2008,
            symbol_allocation=[0,14]
        )


In [9]:
_pusch_config = MyPUSCHConfig()
_pusch_config.tb.mcs_index = 3


_n_size_bwp = _pusch_config.n_size_bwp
_n_start_bwp = _pusch_config.n_start_bwp

_tb_size = _pusch_config.tb_size
_num_coded_bits = _pusch_config.num_coded_bits
_target_coderate = _pusch_config.tb.target_coderate
_num_bits_per_symbol = _pusch_config.tb.num_bits_per_symbol
_num_layers = _pusch_config.num_layers
_n_rnti = _pusch_config.n_rnti
_n_id = _pusch_config.tb.n_id


_binary_source = BinarySource(dtype=tf.float32)

_tb_encoder = TBEncoder(
                target_tb_size=_tb_size,
                num_coded_bits=_num_coded_bits,
                target_coderate=_target_coderate,
                num_bits_per_symbol=_num_bits_per_symbol,
                num_layers=_num_layers,
                n_rnti=_n_rnti,
                n_id=_n_id,
                channel_type="PUSCH", # PUSCHTransmitter
                codeword_index=0, # not supported for PUSCH
                use_scrambler=True,
                verbose=False,
                output_dtype=tf.float32)

_mapper = Mapper("qam", _num_bits_per_symbol, dtype=tf.complex64)

_layer_mapper = LayerMapper(num_layers=_num_layers, dtype=tf.complex64)

_dmrs_length = _pusch_config.dmrs.length
_dmrs_additional_position = _pusch_config.dmrs.additional_position
_num_cdm_groups_without_data = _pusch_config.dmrs.num_cdm_groups_without_data
_n_scid = _pusch_config.dmrs.n_scid
_n_id_n_scid = _pusch_config.dmrs.n_id[0]

_pilot_pattern = PUSCHPilotPattern([_pusch_config],
                                        dtype=tf.complex64)
_mu = 1
_num_ofdm_symbols = 14
_fft_size = 4096
_cyclic_prefix_length = 288
_subcarrier_spacing = 30e3
# _num_guard_subcarriers = (2024, 2024)
_num_guard_subcarriers = (410+_n_start_bwp*12, _fft_size-_n_size_bwp*12-410)
_num_slots_per_frame = 20

# Define the resource grid.
_resource_grid = ResourceGrid(
    num_ofdm_symbols=_num_ofdm_symbols,
    fft_size=_fft_size,
    subcarrier_spacing=_subcarrier_spacing,
    num_tx=1,
    num_streams_per_tx=1,
    cyclic_prefix_length=_cyclic_prefix_length,
    num_guard_carriers=_num_guard_subcarriers,
    dc_null=False,
    pilot_pattern=_pilot_pattern,
    dtype=tf.complex64
)

_resource_grid_mapper = ResourceGridMapper(_resource_grid, dtype=tf.complex64)


_num_tx = 1
_num_rx = 1
_num_tx_ant = 1
_num_rx_ant = 8
_carrier_frequency = 2.55e9  # Carrier frequency in Hz.

_ue_antenna = Antenna(polarization="single",
                        polarization_type="V",
                        antenna_pattern="38.901",
                        carrier_frequency=_carrier_frequency)

_gnb_array = AntennaArray(num_rows=1,
                        num_cols=_num_rx_ant//2,
                        polarization="dual",
                        polarization_type="cross",
                        antenna_pattern="38.901",
                        carrier_frequency=_carrier_frequency)

_channel_model = CDL(model = 'C',
                    delay_spread = 150e-9,
                    carrier_frequency = _carrier_frequency,
                    ut_array = _ue_antenna,
                    bs_array = _gnb_array,
                    direction = 'uplink',
                    min_speed = 10,
                    max_speed = 10)

_channel = OFDMChannel(_channel_model,
                        _resource_grid,
                        normalize_channel=True,
                        return_channel=True)

In [10]:
# name = 'case2081-2'
# tv_dir = f'../Pusch_data/test_vector/{name}'
# os.makedirs(tv_dir, exist_ok=True)
def setCfgReq(dir):
    filename = os.path.join(dir, 'cfgReq.cfg')

    xml_content = f"""<?xml version="1.0"?>

<TestConfig>
	<numSlots>20</numSlots>
	<uliq_car0_ant0>rx_ant_0.bin</uliq_car0_ant0>
	<uliq_car0_ant1>rx_ant_1.bin</uliq_car0_ant1>
	<uliq_car0_ant2>rx_ant_2.bin</uliq_car0_ant2>
	<uliq_car0_ant3>rx_ant_3.bin</uliq_car0_ant3>
	<uliq_car0_ant4>rx_ant_4.bin</uliq_car0_ant4>
	<uliq_car0_ant5>rx_ant_5.bin</uliq_car0_ant5>
	<uliq_car0_ant6>rx_ant_6.bin</uliq_car0_ant6>
	<uliq_car0_ant7>rx_ant_7.bin</uliq_car0_ant7>
	<ul_ref_out>ref.txt</ul_ref_out>
	<start_frame_number>0</start_frame_number>
	<start_slot_number>0</start_slot_number>
</TestConfig>

<ConfigReq>
	<nCarrierIdx>0</nCarrierIdx>
	<nDMRSTypeAPos>3</nDMRSTypeAPos>
	<nPhyCellId>246</nPhyCellId>
	<nDLBandwidth>100</nDLBandwidth>
	<nULBandwidth>100</nULBandwidth>
	<nDLFftSize>4096</nDLFftSize>
	<nULFftSize>4096</nULFftSize>
	<nNrOfTxAnt>8</nNrOfTxAnt>
	<nNrOfRxAnt>8</nNrOfRxAnt>
	<nCarrierAggregationLevel>0</nCarrierAggregationLevel>
	<nFrameDuplexType>1</nFrameDuplexType>
	<nSubcCommon>1</nSubcCommon>
	<nTddPeriod>20</nTddPeriod>
	<sSlotConfig0>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig0>
	<sSlotConfig1>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig1>
	<sSlotConfig2>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig2>
	<sSlotConfig3>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig3>
	<sSlotConfig4>1,1,1,1,1,1,1,1,1,1,1,1,1,1</sSlotConfig4>
	<sSlotConfig5>1,1,1,1,1,1,1,1,1,1,1,1,1,1</sSlotConfig5>
	<sSlotConfig6>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig6>
	<sSlotConfig7>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig7>
	<sSlotConfig8>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig8>
	<sSlotConfig9>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig9>
	<sSlotConfig10>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig10>
	<sSlotConfig11>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig11>
	<sSlotConfig12>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig12>
	<sSlotConfig13>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig13>
	<sSlotConfig14>1,1,1,1,1,1,1,1,1,1,1,1,1,1</sSlotConfig14>
	<sSlotConfig15>1,1,1,1,1,1,1,1,1,1,1,1,1,1</sSlotConfig15>
	<sSlotConfig16>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig16>
	<sSlotConfig17>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig17>
	<sSlotConfig18>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig18>
	<sSlotConfig19>2,2,2,2,2,2,2,2,2,2,2,2,2,2</sSlotConfig19>
	<nCyclicPrefix>0</nCyclicPrefix>
</ConfigReq>

<RxConfig>
	<SlotNum0>null.cfg</SlotNum0>
	<SlotNum1>null.cfg</SlotNum1>
	<SlotNum2>null.cfg</SlotNum2>
	<SlotNum3>null.cfg</SlotNum3>
	<SlotNum4>slot4.cfg</SlotNum4>
	<SlotNum5>slot5.cfg</SlotNum5>
	<SlotNum6>null.cfg</SlotNum6>
	<SlotNum7>null.cfg</SlotNum7>
	<SlotNum8>null.cfg</SlotNum8>
	<SlotNum9>null.cfg</SlotNum9>
	<SlotNum10>null.cfg</SlotNum10>
	<SlotNum11>null.cfg</SlotNum11>
	<SlotNum12>null.cfg</SlotNum12>
	<SlotNum13>null.cfg</SlotNum13>
	<SlotNum14>slot14.cfg</SlotNum14>
	<SlotNum15>slot15.cfg</SlotNum15>
	<SlotNum16>null.cfg</SlotNum16>
	<SlotNum17>null.cfg</SlotNum17>
	<SlotNum18>null.cfg</SlotNum18>
	<SlotNum19>null.cfg</SlotNum19>
</RxConfig>
"""
    with open(filename, 'w') as file:
        file.write(xml_content)

# setCfgReq(tv_dir)

In [11]:
def setUlCfgReq(slotcfg, dir):
    filename = os.path.join(dir, f'slot{slotcfg.nSlot}.cfg')

    # Define the XML structure dynamically
    xml_content = f"""<?xml version="1.0"?>

<Ul_Config_Req>

	<UlConfigReqL1L2Header>
		<nSFN>{slotcfg.nSFN}</nSFN>
		<nSlot>{slotcfg.nSlot}</nSlot>
		<nPDU>{slotcfg.nPDU}</nPDU>
		<nGroup>{slotcfg.nGroup}</nGroup>
		<nUlsch>{slotcfg.nUlsch}</nUlsch>
		<nUlcch>{slotcfg.nUlcch}</nUlcch>
		<nRachPresent>{slotcfg.nRachPresent}</nRachPresent>
	</UlConfigReqL1L2Header>

	<UL_SCH_PDU0>
		<nRNTI>{slotcfg.nRNTI}</nRNTI>
		<nUEId>0</nUEId>
		<nBWPSize>{slotcfg.nBWPSize}</nBWPSize>
		<nBWPStart>{slotcfg.nBWPStart}</nBWPStart>
		<nSubcSpacing>{slotcfg.nSubcSpacing}</nSubcSpacing>
		<nCpType>{slotcfg.nCpType}</nCpType>
		<nULType>{slotcfg.nULType}</nULType>
		<nMcsTable>{slotcfg.nMcsTable}</nMcsTable>
		<nMCS>{slotcfg.nMCS}</nMCS>
		<nTransPrecode>{slotcfg.nTransPrecode}</nTransPrecode>
		<nTransmissionScheme>{slotcfg.nTransmissionScheme}</nTransmissionScheme>
		<nNrOfLayers>{slotcfg.nNrOfLayers}</nNrOfLayers>
		<nPortIndex0>{slotcfg.nPortIndex[0]}</nPortIndex0>
		<nNid>{slotcfg.nNid}</nNid>
		<nSCID>{slotcfg.nSCID}</nSCID>
		<nNIDnSCID>{slotcfg.nNIDnSCID}</nNIDnSCID>
		<nNrOfAntennaPorts>{slotcfg.nNrOfAntennaPorts}</nNrOfAntennaPorts>
		<nVRBtoPRB>{slotcfg.nVRBtoPRB}</nVRBtoPRB>
		<nPMI>{slotcfg.nPMI}</nPMI>
		<nStartSymbolIndex>{slotcfg.nStartSymbolIndex}</nStartSymbolIndex>
		<nNrOfSymbols>{slotcfg.nNrOfSymbols}</nNrOfSymbols>
		<nResourceAllocType>{slotcfg.nResourceAllocType}</nResourceAllocType>
		<nRBStart>{slotcfg.nRBStart}</nRBStart>
		<nRBSize>{slotcfg.nRBSize}</nRBSize>
		<nTBSize>{slotcfg.nTBSize}</nTBSize>
		<nRV>{slotcfg.nRV}</nRV>
		<nHARQID>{slotcfg.nHARQID}</nHARQID>
		<nNDI>{slotcfg.nNDI}</nNDI>
		<nMappingType>{slotcfg.nMappingType}</nMappingType>
		<nDMRSConfigType>{slotcfg.nDMRSConfigType}</nDMRSConfigType>
		<nNrOfCDMs>{slotcfg.nNrOfCDMs}</nNrOfCDMs>
		<nNrOfDMRSSymbols>{slotcfg.nNrOfDMRSSymbols}</nNrOfDMRSSymbols>
		<nDMRSAddPos>{slotcfg.nDMRSAddPos}</nDMRSAddPos>
		<nPTRSPresent>{slotcfg.nPTRSPresent}</nPTRSPresent>
		<nAck>{slotcfg.nAck}</nAck>
		<nAlphaScaling>{slotcfg.nAlphaScaling}</nAlphaScaling>
		<nBetaOffsetACKIndex>{slotcfg.nBetaOffsetACKIndex}</nBetaOffsetACKIndex>
		<nCsiPart1>{slotcfg.nCsiPart1}</nCsiPart1>
		<nBetaOffsetCsiPart1Index>{slotcfg.nBetaOffsetCsiPart1Index}</nBetaOffsetCsiPart1Index>
		<nCsiPart2>{slotcfg.nCsiPart2}</nCsiPart2>
		<nBetaOffsetCsiPart2Index>{slotcfg.nBetaOffsetCsiPart2Index}</nBetaOffsetCsiPart2Index>
		<nTpPi2BPSK>{slotcfg.nTpPi2BPSK}</nTpPi2BPSK>
		<nTPPuschID>{slotcfg.nTPPuschID}</nTPPuschID>
		<nRxRUIdx0>{slotcfg.nRxRUIdx[0]}</nRxRUIdx0>
		<nRxRUIdx1>{slotcfg.nRxRUIdx[1]}</nRxRUIdx1>
		<nRxRUIdx2>{slotcfg.nRxRUIdx[2]}</nRxRUIdx2>
		<nRxRUIdx3>{slotcfg.nRxRUIdx[3]}</nRxRUIdx3>
		<nRxRUIdx4>{slotcfg.nRxRUIdx[4]}</nRxRUIdx4>
		<nRxRUIdx5>{slotcfg.nRxRUIdx[5]}</nRxRUIdx5>
		<nRxRUIdx6>{slotcfg.nRxRUIdx[6]}</nRxRUIdx6>
		<nRxRUIdx7>{slotcfg.nRxRUIdx[7]}</nRxRUIdx7>
	</UL_SCH_PDU0>

	<PUSCH_GROUP_INFO0>
		<nUE>{slotcfg.nUE}</nUE>
		<nPduIdx0>{slotcfg.nPduIdx[0]}</nPduIdx0>
	</PUSCH_GROUP_INFO0>

</Ul_Config_Req>
"""
    with open(filename, 'w') as file:
        file.write(xml_content)

# for i,slot in enumerate([4,5,14,15]):
# 	setUlCfgReq(df.iloc[i], tv_dir)


In [12]:
def set_null(dir):
    filename = os.path.join(dir, 'null.cfg')


    xml_content = f"""<?xml version="1.0"?>

<Ul_Config_Req>
	<UlConfigReqL1L2Header>
		<nSFN>0</nSFN>
		<nSlot>0</nSlot>
		<nPDU>0</nPDU>
		<nGroup>0</nGroup>
		<nUlsch>0</nUlsch>
		<nUlcch>0</nUlcch>
		<nRachPresent>0</nRachPresent>
	</UlConfigReqL1L2Header>
</Ul_Config_Req>
"""
    with open(filename, 'w') as file:
        file.write(xml_content)

# set_null(tv_dir)


In [13]:
# xs = tf.Variable(tf.zeros([1, 1, 1, 20, 14, 3276], dtype=tf.complex64))
rxSigFreq = tf.Variable(tf.zeros([1, 1, 8, 20, 14, 3276], dtype=tf.complex64))
inBits = tf.Variable(tf.zeros([1, 1, 20, 288], dtype=tf.float32))


for i in [4,5,14,15]:
    b = tf.constant(generate_prng_seq(_tb_size, 2004*10+i).reshape(1,1,-1), dtype=tf.float32)
    # b = _binary_source([batch_size, _num_tx, _tb_size])
    c = _tb_encoder(b)
    x_map = _mapper(c)
    x_layer = _layer_mapper(x_map)

    pusch_config = _pusch_config.clone()
    pusch_config.carrier.slot_number = i
    pilot_pattern = PUSCHPilotPattern([pusch_config],
                                            dtype=tf.complex64)

    _resource_grid_mapper._resource_grid.pilot_pattern = pilot_pattern
    x = _resource_grid_mapper(x_layer)
    
    # xs[:,:,:,i,:,:].assign(x[...,410:-410])
    inBits[:,:,i,:].assign(b)

    y = tf.repeat(x, 8, 2)
    # y, h = _channel([x, 0.1])

    rxSigFreq[:,:,:,i,:,:].assign(y[...,410:-410])

# rxSigFreq = tf.Variable(tf.zeros([1,1,8,20,14,3276], dtype=tf.complex64))
# rxSigFreq.assign(rxSigFreq)

# inBits = tf.Variable(tf.zeros([1, 1, 20, 19464], dtype=tf.float32))
# inBits.assign(inBits)
rxSigFreq

<tf.Variable 'Variable:0' shape=(1, 1, 8, 20, 14, 3276) dtype=complex64, numpy=
array([[[[[[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
           ...,
           [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

          [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
           ...,
           [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

          [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j

In [14]:
tf.math.reduce_sum(rxSigFreq[0,0], [0,1,3])

<tf.Tensor: shape=(14,), dtype=complex64, numpy=
array([-1.81019501e+02+6.78823013e+01j,  4.52548599e+01+4.52548637e+01j,
        8.00000000e+01+1.44000000e+02j, -2.26274185e+01-4.52548599e+01j,
       -1.35764618e+02+1.13137184e+02j,  2.26274223e+01+2.26497650e-06j,
        2.26497650e-06-1.58392059e+02j, -2.26274185e+01+9.05097427e+01j,
       -2.26497650e-06+9.05097427e+01j, -4.52548599e+01-4.52548599e+01j,
       -1.13137184e+02-2.26274223e+01j, -8.00000000e+01+0.00000000e+00j,
        2.26274204e+01+1.35764618e+02j, -9.05097427e+01+2.26274242e+01j],
      dtype=complex64)>

In [15]:
# xs = tf.Variable(tf.zeros([1, 1, 1, 20, 14, 3276], dtype=tf.complex64))
# ys = tf.Variable(tf.zeros([1, 1, 8, 20, 14, 3276], dtype=tf.complex64))
# bs = tf.Variable(tf.zeros([1, 1, 20, 19464], dtype=tf.float32))

# df = pd.read_parquet('/workspaces/thanh/Pusch_data/parquet/20250224082926279828.parquet',engine='pyarrow')

# for n,i in enumerate([4,5,14,15]):
#     with open(f'../Pusch_data/pickle/{df.iloc[n].Data_dirname}/{df.iloc[n].Data_filename}.y.pkl', 'rb') as file:
#         y = pickle.load(file)
#     ys[:,:,:,i,:,:].assign(y[...,410:-410])

#     with open(f'../Pusch_data/pickle/{df.iloc[n].Data_dirname}/{df.iloc[n].Data_filename}.b.pkl', 'rb') as file:
#         b = pickle.load(file)
#     bs[:,:,i,:].assign(b)
    

# rxSigFreq = tf.Variable(tf.zeros([1,1,8,20,14,3276], dtype=tf.complex64))
# # for i in range(8):
# rxSigFreq[:,:,:,:,:,:].assign(ys)
# inBits.assign(bs)

In [41]:
def setRxData(rxSigFreq, dir):
    # rxSigFreq = tf.reshape(rxSigFreq, -1)
    rxSigFreq = tf.stack((tf.math.real(rxSigFreq), tf.math.imag(rxSigFreq)), axis=-1)
    
    rxSigFreq = rxSigFreq/tf.math.reduce_max(tf.abs(rxSigFreq))

    rxSigFreq = tf.cast(tf.round(rxSigFreq*2**13), tf.int16)
    rxSigFreq = tf.reshape(rxSigFreq,[8,-1])
    
    for rxIdx in range(8):
        file_path = os.path.join(dir, f'rx_ant_{rxIdx}.bin')
        with open(file_path, 'wb') as file:
            file.write(rxSigFreq[rxIdx])
    return rxSigFreq
# setRxData(rxSigFreq[0,0], tv_dir)

In [ ]:
def setRef(inBits, dir):
    filename = os.path.join(dir, 'ref.txt')
    with open(filename, 'w') as file:
        file.write(f'##----------------------------------------------------------------------------\n')
        fn = -1
        for n,payload in enumerate(inBits):
            # payload = inBits[:, i]
            if n in [4,5,14,15]:
                fn = fn + 1
                tbSize = payload.shape[-1]//8
                file.write(f'#type[PUSCH] fn[{fn}] slot[{n}] sym[0] carrier[0] chanId[0] len[{tbSize}]\n')
                file.write(f'\t  #ta[0] cqi[0.0] stat[1]\n')
                file.write(f'\t  #data[\n')
                Q = tbSize//64
                payload = tf.math.reduce_sum(tf.reshape(payload,[-1, 8]) * tf.constant([[128, 64, 32, 16, 8, 4, 2, 1]], dtype=tf.uint8), axis=1)
                # print(payload.shape, Q, r)
                for q in range(Q):
                    file.write('\t        ')
                    for r in range(64):
                        file.write(f'{payload[64*q + r]:3d}, ')
                    file.write('\n')
                file.write('\t        ')
                for r in range(tbSize%64-1):
                    file.write(f'{payload[64*Q + r]:3d}, ')
                file.write(f'{payload[-1]:3d}\n')
                file.write(f'\t       ]\n')
                file.write(f'------------------------------------------------------------------------------\n')
            else: 
                file.write(f'##----------------------------------------------------------------------------\n')
# setRef(inBits[0,0], tv_dir)


In [56]:
df = pd.read_parquet('/workspaces/thanh/Pusch_data/parquet/4RB_test.parquet')
df

,nSFN,nSlot,nPDU,nGroup,nUlsch,nUlcch,nRachPresent,nRNTI,nUEId,nBWPSize,...,nTPPuschID,nRxRUIdx,nUE,nPduIdx,Channel_model,Speed,Delay_spread,Esno_db,Data_filename,Data_dirname
0,0,4,1,1,1,0,0,20002,0,273,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Normalized-CDL-B,1,1.000000e-08,5.0,20250228095232898892,4RB_test
1,0,5,1,1,1,0,0,20002,0,273,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Normalized-CDL-B,1,1.000000e-08,5.0,20250228095233336912,4RB_test
2,0,14,1,1,1,0,0,20002,0,273,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Normalized-CDL-B,1,1.000000e-08,5.0,20250228095233851763,4RB_test
3,0,15,1,1,1,0,0,20002,0,273,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Normalized-CDL-B,1,1.000000e-08,5.0,20250228095234204174,4RB_test


In [57]:
df.iloc[0]

nSFN                                               0
nSlot                                              4
nPDU                                               1
nGroup                                             1
nUlsch                                             1
nUlcch                                             0
nRachPresent                                       0
nRNTI                                          20002
nUEId                                              0
nBWPSize                                         273
nBWPStart                                          0
nSubcSpacing                                       1
nCpType                                            0
nULType                                            0
nMcsTable                                          0
nMCS                                               3
nTransPrecode                                      0
nTransmissionScheme                                0
nNrOfLayers                                   

In [58]:
arrays = np.array_split(df.to_numpy(), len(df) // 4)
# Convert back to DataFrames
dfs = [pd.DataFrame(arr, columns=df.columns) for arr in arrays]
len(dfs)

1

In [59]:
tv_dir = '../Pusch_data/4RB_test'
for df_n, df_i in enumerate(dfs):
    # print(df_i)
    gr_dir = f'{tv_dir}/case20002/group-{df_n+1}'
    os.makedirs(gr_dir, exist_ok=True)
    setCfgReq(gr_dir)
    for n,i in enumerate([4,5,14,15]):
        setUlCfgReq(df_i.iloc[n], gr_dir)
    set_null(gr_dir)
    

    rxSigFreq = tf.Variable(tf.zeros([1, 8, 20, 14, 3276], dtype=tf.complex64))
    inBits = 20*[None]

    for n,i in enumerate([4,5,14,15]):
        with open(f'../Pusch_data/pickle/{df_i.iloc[n].Data_dirname}/{df_i.iloc[n].Data_filename}.y.pkl', 'rb') as file:
            y = pickle.load(file)
        print(y.shape)
        rxSigFreq[:,:,i,:,12*df_i.iloc[n].nRBStart:12*(df_i.iloc[n].nRBStart + df_i.iloc[n].nRBSize)].assign(y)

        with open(f'../Pusch_data/pickle/{df_i.iloc[n].Data_dirname}/{df_i.iloc[n].Data_filename}.b.pkl', 'rb') as file:
            b = pickle.load(file)
        # inBits[:,i,:].assign(b)
        print(np.sum(b))
        inBits[i] = b
    print(inBits)
    setRxData(rxSigFreq, gr_dir)
    setRef(inBits, gr_dir)

    # print(tf.math.reduce_min(rxSigFreq), tf.math.reduce_max(rxSigFreq))

    

(1, 8, 14, 48)
148
(1, 8, 14, 48)
133
(1, 8, 14, 48)
147
(1, 8, 14, 48)
158
[None, None, None, None, <tf.Tensor: shape=(1, 288), dtype=uint8, numpy=
array([[0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
        1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 0, 1,